In [1]:
run_checks = False

### Overview
This notebook works on the IEEE-CIS Fraud Detection competition. Here I build a simple XGBoost model based on a balanced dataset.

### Lessons:

. keep the categorical variables as single items

. Use a high max_depth for xgboost (maybe 40)


### Ideas to try:

. train divergence of expected value (eg. for TransactionAmt and distance based on the non-fraud subset (not all subset as in the case now)

. try using a temporal approach to CV

In [2]:
# all imports necessary for this notebook
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import gc
import copy
import missingno as msno 
import xgboost
from xgboost import XGBClassifier, XGBRegressor
from sklearn.model_selection import StratifiedKFold, cross_validate, train_test_split 
from sklearn.metrics import roc_auc_score, r2_score

import warnings
warnings.filterwarnings('ignore')

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/master-df-time-adjusted-top-100csv/master_df_time_adjusted_top_100.csv
/kaggle/input/ieee-fraud-detection/train_identity.csv
/kaggle/input/ieee-fraud-detection/train_transaction.csv
/kaggle/input/ieee-fraud-detection/test_transaction.csv
/kaggle/input/ieee-fraud-detection/test_identity.csv
/kaggle/input/ieee-fraud-detection/sample_submission.csv
/kaggle/input/ieee-preprocessed/master_df_top_300.csv
/kaggle/input/ieee-preprocessed/master_df_top_200.csv
/kaggle/input/ieee-preprocessed/master_df_top_100.csv
/kaggle/input/ieee-preprocessed/master_df_top_all.csv


In [3]:
# Helpers
    
def seed_everything(seed=0):
    '''Seed to make all processes deterministic '''
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    
def drop_correlated_cols(df, threshold, cols_to_keep, sample_frac = 1):
    '''Drops one of two dataframe's columns whose pairwise pearson's correlation is above the provided threshold'''
    if sample_frac != 1:
        dataset = df.sample(frac = sample_frac).copy()
    else:
        dataset = df
        
    col_corr = set() # Set of all the names of deleted columns
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        if corr_matrix.columns[i] in col_corr:
            continue
        for j in range(i):
            if corr_matrix.columns[j] in cols_to_keep:
                continue
            if (corr_matrix.iloc[i, j] >= threshold) and (corr_matrix.columns[j] not in col_corr):
                colname = corr_matrix.columns[i] # getting the name of column
                col_corr.add(colname)
    del dataset
    gc.collect()
    df.drop(columns = col_corr, inplace = True)

def calc_feature_difference(df, feature_name, indep_features, min_r2 = 0.1, min_r2_improv = 0, frac1 = 0.1,
                              max_depth_start = 2, max_depth_step = 4):
    
    from copy import deepcopy
    
    print("Feature name %s" %feature_name)
    #print("Indep_features %s" %indep_features)
    
    is_imrpoving = True
    curr_max_depth = max_depth_start
    best_r2 = float("-inf")
    clf_best = np.nan
    
    while is_imrpoving:
        clf = XGBRegressor(max_depth = curr_max_depth)

        rand_sample_indeces = df[df[feature_name].notnull()].sample(frac = frac1).index
        clf.fit(df.loc[rand_sample_indeces, indep_features], df.loc[rand_sample_indeces, feature_name]) 

        rand_sample_indeces = df[df[feature_name].notnull()].sample(frac = frac1).index
        
        pred_y = clf.predict(df.loc[rand_sample_indeces, indep_features])
        r2Score = r2_score(df.loc[rand_sample_indeces, feature_name], pred_y)
        print("%d, R2 score %.4f" % (curr_max_depth, r2Score))
        
        curr_max_depth = curr_max_depth + max_depth_step
        
        if r2Score > best_r2:
            best_r2 = r2Score
            clf_best = deepcopy(clf)
        if r2Score < best_r2 + (best_r2 * min_r2_improv) or (curr_max_depth > max_depth_start * max_depth_step and best_r2 < min_r2 / 2):
            is_imrpoving = False

    print("The best R2 score of %.4f" % ( best_r2))
    
    if best_r2 > min_r2:
        pred_feature = clf_best.predict(df.loc[:, indep_features])
        return (df[feature_name] - pred_feature)
    else:
        return df[feature_name]

In [4]:
seed_everything()
pd.set_option('display.max_columns', 500)

In [5]:
master_df = pd.read_csv('/kaggle/input/ieee-preprocessed/master_df_top_100.csv')
master_df.head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C5,C6,C8,C9,C10,C11,C13,C14,D1,D2,D3,D4,D5,D8,D9,D10,D11,D14,D15,M3,M4,M5,M6,M7,M8,M9,V4,V12,V20,V35,V36,V53,V56,V62,V70,V75,V76,V82,V87,V91,V94,V99,V102,V103,V112,V126,V127,V128,V130,V131,V133,V141,V145,V149,V152,V162,V173,V176,V177,V188,V189,V191,V201,V206,V208,V209,V223,V230,V244,V245,V254,V257,V258,V269,V271,V283,V285,V294,V304,V306,V307,V308,V310,V312,V313,V314,V315,V317,V320,V324,V329,id_01,id_02,id_05,id_06,id_13,id_17,id_19,id_20,id_26,DeviceInfo,is_train_df,P_emaildomain_2,P_emaildomain_3,id_31_edge,id_31_chrome_version_newness,id_31_safari_version_newness,id_30_android,id_33_resolution,TransactionAmt_decimal,weekday,hours,card3_144.0,card3_150.0,card3_185.0,R_emaildomain_2_com,id_14_-420.0,id_14_60.0,M4_M0,M4_M1,card6_credit,M8_F,hours_5.0,hours_11.0,card4_american express,R_emaildomain_anonymous.com,addr2_87.0,M6_F,M3_F,M5_F,card1_9500,card1_9633,card1_17188,card1_infrequent_category,P_emaildomain_comcast.net,P_emaildomain_gmail.com,P_emaildomain_hotmail.com,P_emaildomain_me.com,P_emaildomain_msn.com,P_emaildomain_optonline.net,P_emaildomain_verizon.net,P_emaildomain_infrequent_category,addr1_204.0,addr1_299.0,addr1_325.0,addr1_337.0,addr1_472.0,addr1_485.0,addr1_infrequent_category,ProductCD_H,ProductCD_R,id_20_333.0,id_20_549.0,id_20_612.0,card2_225.0,card2_268.0,card2_321.0,card2_481.0,card2_490.0,card2_553.0,card2_555.0,card2_567.0,card2_infrequent_category,M7_F,weekday_1.0,weekday_2.0,weekday_3.0,weekday_4.0,weekday_5.0,card5_166.0,card5_224.0,P_emaildomain_1_infrequent_category,id_17_166.0,id_32_24.0
0,2987000,0.0,86400,68.5,W,13926,NaN,150.0,discover,142.0,credit,315.0,87.0,19.0,NaN,NaN,NaN,1.0,1.0,0.0,1.0,0.0,1.0,0.0,2.0,1.0,1.0,14.0,NaN,13.0,NaN,NaN,NaN,NaN,13.0,13.0,NaN,0.0,T,M2,F,T,NaN,NaN,NaN,1.0,1.0,1.0,NaN,NaN,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,117.0,0.0,0.0,0.0,117.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,1.0,0.0,0.0,117.0,0.0,0.0,0.0,0.0,0.0,0.0,117.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,500,0.0,0.0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2987001,0.0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,credit,325.0,87.0,NaN,NaN,gmail.com,NaN,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,M0,T,T,NaN,NaN,NaN,NaN,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,com,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,0.0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2,2987002,0.0,86469,59.0,W,4663,490.0,150.0,visa,166.0,debit,330.0,87.0,287.0,NaN,outlook.com,NaN,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,0.0,315.0,NaN,315.0,T,M0,F,F,F,F,F,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,com,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,0.0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0
3,2987003,0.0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,debit,476.0,87.0,NaN,NaN,yahoo.com,NaN,2.0,5.0,0.0,4.0,0.0,1.0,0.0,1.0,25.0,1.0,112.0,112.0,0.0,94.0,0.0,NaN,NaN,84.0,NaN,NaN,111.0,NaN,M0,T,F,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,10.0,38.0,2

In [6]:
master_df_time_adjusted = pd.read_csv('/kaggle/input/master-df-time-adjusted-top-100csv/master_df_time_adjusted_top_100.csv')
master_df_time_adjusted.head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C5,C6,C8,C9,C11,C13,C14,D1,D2,D3,D4,D5,D8,D10,D11,D14,D15,M3,M4,M5,M6,M7,M8,M9,V4,V12,V20,V35,V36,V53,V56,V62,V70,V75,V76,V82,V87,V91,V99,V112,V126,V127,V128,V130,V131,V133,V141,V145,V149,V152,V162,V173,V176,V177,V188,V189,V191,V201,V206,V208,V209,V223,V230,V244,V245,V254,V257,V258,V269,V271,V283,V285,V304,V306,V307,V310,V312,V313,V314,V315,V320,id_01,id_02,id_05,id_06,id_13,id_17,id_19,id_20,id_26,DeviceInfo,is_train_df,P_emaildomain_2,P_emaildomain_3,id_31_edge,id_31_chrome_version_newness,id_31_safari_version_newness,id_30_android,id_33_resolution,TransactionAmt_decimal,weekday,hours,R_emaildomain_2_com,id_14_-420.0,id_14_60.0,M4_M0,M4_M1,M8_F,R_emaildomain_anonymous.com,M6_F,M3_F,M5_F,P_emaildomain_comcast.net,P_emaildomain_gmail.com,P_emaildomain_hotmail.com,P_emaildomain_me.com,P_emaildomain_msn.com,P_emaildomain_optonline.net,P_emaildomain_verizon.net,P_emaildomain_infrequent_category,id_20_333.0,id_20_549.0,id_20_612.0,M7_F,P_emaildomain_1_infrequent_category,id_32_24.0
0,2987000,0.0,0,68.5,4,12696,-1,50,1,42,1,215,77,19.0,NaN,NaN,NaN,0.452748,-56.367245,0.0,-35.609619,0.147989,1.0,-7.334560,-76.363113,-4.731055,-7.852631,NaN,13.0,NaN,NaN,NaN,-13.998543,13.0,NaN,-219.237869,T,M2,F,T,NaN,NaN,NaN,1.0,0.332354,1.0,NaN,NaN,0.311933,1.0,1.0,-0.696159,0.420110,0.371908,0.0,-0.400271,-0.477729,-5.658206,1.0,-442.928955,-6130.065430,-47.239407,0.0,0.0,93.224512,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,-2.146344,0.000325,-98.388420,-65.309341,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.971710,0,0,-0.015990,-0.019621,0,-0.265431,0,-0.164929,-0.003047,-0.080853,0,0.437072,0,0,-0.025767,0,0,0,0,0,-0.006322,-0.000008,0,-0.390713,0,-0.016675
1,2987001,0.0,1,29.0,4,1726,304,50,2,2,1,225,77,NaN,NaN,gmail.com,NaN,-3.620259,-56.254463,0.0,-4.153918,-0.184928,0.0,-9.372200,-2.555614,-0.118743,-88.157257,NaN,NaN,0.0,NaN,NaN,-157.705399,NaN,NaN,-163.875885,NaN,M0,T,T,NaN,NaN,NaN,NaN,-0.726845,1.0,-0.738347,-0.600466,-0.845680,1.0,1.0,-0.669546,-0.728800,-0.656142,1.0,-0.179304,-0.418249,-7.665982,1.0,-29.248983,-1255.726807,10.947166,0.0,0.0,294.409943,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,-0.674220,0.007776,-1461.669067,325.092743,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,com,NaN,NaN,NaN,NaN,NaN,NaN,-256.136840,0,0,-0.000335,0.000125,0,0.185291,0,-0.182496,0.005825,-0.163323,0,-0.526690,0,1,0.002152,0,0,0,0,0,-0.154146,-0.000008,0,-0.221700,0,-0.005720
2,2987002,0.0,2,59.0,4,3597,390,50,3,66,2,230,77,287.0,NaN,outlook.com,NaN,-4.552312,-57.456963,0.0,-5.875078,-0.109141,1.0,-13.440516,-8.605875,-3.505404,-48.127869,NaN,NaN,0.0,NaN,NaN,-70.123619,315.0,NaN,91.240555,T,M0,F,F,F,F,F,1.0,0.299800,1.0,0.297722,0.394673,0.299507,1.0,1.0,-0.521132,0.398391,0.318474,1.0,0.029664,-0.472105,-8.090937,1.0,-14.131649,-8801.166016,-127.531815,0.0,0.0,47.391029,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,-0.530957,0.000325,-206.973801,-566.554932,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,com,NaN,NaN,NaN,NaN,NaN,NaN,-366.548200,0,0,0.001292,0.000063,0,0.431426,0,0.819766,0.029431,0.422267,0,0.758143,0,0,0.015687,0,0,0,0,0,-0.000358,-0.000008,0,0.765233,0,0.001808
3,2987003,0.0,3,50.0,4,16830,467,50,2,17,2,376,77,NaN,NaN,yahoo.com,NaN,-28.246696,-45.159729,0.0,-2.702492,0.607816,1.0,-11.248108,-48.973129,4.087717,-83.858826,112.0,0.0,94.0,0.0,NaN,-13.188324,NaN,NaN,-202.706940,NaN,M0,T,F,NaN,NaN,NaN,NaN,0.284040,1.0,0.588696,0.456558,0.216863,1.0,1.0,-0.823855,0.633004,0.214649,1.0,0.066225,-0.269714,0.613984,1.0,-16.667755,-7303.942383,875.265041,354.0,135.0,1494.660744,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [7]:
master_df_time_adjusted.drop(columns = [ 'hours', 'weekday', 'TransactionDT', 'ProductCD', 'card1', 'card2', 'card3', 'card4', 'card5'
                                                      , 'card6', 'addr1', 'addr2', 'is_train_df', 'isFraud'], inplace = True)

In [8]:
master_df_merged = pd.merge(master_df, master_df_time_adjusted, on='TransactionID', how='left', validate = '1:1')

In [9]:
master_df_merged.shape

(1097231, 323)

In [10]:
master_df_merged.head()

,TransactionID,isFraud,TransactionDT,TransactionAmt_x,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1_x,dist2_x,P_emaildomain_x,R_emaildomain_x,C1_x,C2_x,C5_x,C6_x,C8_x,C9_x,C10,C11_x,C13_x,C14_x,D1_x,D2_x,D3_x,D4_x,D5_x,D8_x,D9,D10_x,D11_x,D14_x,D15_x,M3_x,M4_x,M5_x,M6_x,M7_x,M8_x,M9_x,V4_x,V12_x,V20_x,V35_x,V36_x,V53_x,V56_x,V62_x,V70_x,V75_x,V76_x,V82_x,V87_x,V91_x,V94,V99_x,V102,V103,V112_x,V126_x,V127_x,V128_x,V130_x,V131_x,V133_x,V141_x,V145_x,V149_x,V152_x,V162_x,V173_x,V176_x,V177_x,V188_x,V189_x,V191_x,V201_x,V206_x,V208_x,V209_x,V223_x,V230_x,V244_x,V245_x,V254_x,V257_x,V258_x,V269_x,V271_x,V283_x,V285_x,V294,V304_x,V306_x,V307_x,V308,V310_x,V312_x,V313_x,V314_x,V315_x,V317,V320_x,V324,V329,id_01_x,id_02_x,id_05_x,id_06_x,id_13_x,id_17_x,id_19_x,id_20_x,id_26_x,DeviceInfo_x,is_train_df,P_emaildomain_2_x,P_emaildomain_3_x,id_31_edge_x,id_31_chrome_version_newness_x,id_31_safari_version_newness_x,id_30_android_x,id_33_resolution_x,TransactionAmt_decimal_x,weekday,hours,card3_144.0,card3_150.0,card3_185.0,R_emaildomain_2_com_x,id_14_-420.0_x,id_14_60.0_x,M4_M0_x,M4_M1_x,card6_credit,M8_F_x,hours_5.0,hours_11.0,card4_american express,R_emaildomain_anonymous.com_x,addr2_87.0,M6_F_x,M3_F_x,M5_F_x,card1_9500,card1_9633,card1_17188,card1_infrequent_category,P_emaildomain_comcast.net_x,P_emaildomain_gmail.com_x,P_emaildomain_hotmail.com_x,P_emaildomain_me.com_x,P_emaildomain_msn.com_x,P_emaildomain_optonline.net_x,P_emaildomain_verizon.net_x,P_emaildomain_infrequent_category_x,addr1_204.0,addr1_299.0,addr1_325.0,addr1_337.0,addr1_472.0,addr1_485.0,addr1_infrequent_category,ProductCD_H,ProductCD_R,id_20_333.0_x,id_20_549.0_x,id_20_612.0_x,card2_225.0,card2_268.0,card2_321.0,card2_481.0,card2_490.0,card2_553.0,card2_555.0,card2_567.0,card2_infrequent_category,M7_F_x,weekday_1.0,weekday_2.0,weekday_3.0,weekday_4.0,weekday_5.0,card5_166.0,card5_224.0,P_emaildomain_1_infrequent_category_x,id_17_166.0,id_32_24.0_x,TransactionAmt_y,dist1_y,dist2_y,P_emaildomain_y,R_emaildomain_y,C1_y,C2_y,C5_y,C6_y,C8_y,C9_y,C11_y,C13_y,C14_y,D1_y,D2_y,D3_y,D4_y,D5_y,D8_y,D10_y,D11_y,D14_y,D15_y,M3_y,M4_y,M5_y,M6_y,M7_y,M8_y,M9_y,V4_y,V12_y,V20_y,V35_y,V36_y,V53_y,V56_y,V62_y,V70_y,V75_y,V76_y,V82_y,V87_y,V91_y,V99_y,V112_y,V126_y,V127_y,V128_y,V130_y,V131_y,V133_y,V141_y,V145_y,V149_y,V152_y,V162_y,V173_y,V176_y,V177_y,V188_y,V189_y,V191_y,V201_y,V206_y,V208_y,V209_y,V223_y,V230_y,V244_y,V245_y,V254_y,V257_y,V258_y,V269_y,V271_y,V283_y,V285_y,V304_y,V306_y,V307_y,V310_y,V312_y,V313_y,V314_y,V315_y,V320_y,id_01_y,id_02_y,id_05_y,id_06_y,id_13_y,id_17_y,id_19_y,id_20_y,id_26_y,DeviceInfo_y,P_emaildomain_2_y,P_emaildomain_3_y,id_31_edge_y,id_31_chrome_version_newness_y,id_31_safari_version_newness_y,id_30_android_y,id_33_resolution_y,TransactionAmt_decimal_y,R_emaildomain_2_com_y,id_14_-420.0_y,id_14_60.0_y,M4_M0_y,M4_M1_y,M8_F_y,R_emaildomain_anonymous.com_y,M6_F_y,M3_F_y,M5_F_y,P_emaildomain_comcast.net_y,P_emaildomain_gmail.com_y,P_emaildomain_hotmail.com_y,P_emaildomain_me.com_y,P_emaildomain_msn.com_y,P_emaildomain_optonline.net_y,P_emaildomain_verizon.net_y,P_emaildomain_infrequent_category_y,id_20_333.0_y,id_20_549.0_y,id_20_612.0_y,M7_F_y,P_emaildomain_1_infrequent_category_y,id_32_24.0_y
0,2987000,0.0,86400,68.5,W,13926,NaN,150.0,discover,142.0,credit,315.0,87.0,19.0,NaN,NaN,NaN,1.0,1.0,0.0,1.0,0.0,1.0,0.0,2.0,1.0,1.0,14.0,NaN,13.0,NaN,NaN,NaN,NaN,13.0,13.0,NaN,0.0,T,M2,F,T,NaN,NaN,NaN,1.0,1.0,1.0,NaN,NaN,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,117.0,0.0,0.0,0.0,117.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,1.0,0.0,0.0,117.0,0.0,0.0,0.0,0.0,0.0,0.0,117.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,500,0.0,0.0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,68.5,19.0,NaN,NaN,NaN,0.452748,-56.367245,0.0,-35.609619,0.147989,1.0,-7.3345

In [11]:
del master_df, master_df_time_adjusted
gc.collect()

0

In [12]:
cols_to_keep = ['TransactionID', 'hours', 'weekday', 'TransactionDT', 'ProductCD', 'card1', 'card2', 'card3', 'card4', 'card5'
                                                      , 'card6', 'addr1', 'addr2', 'is_train_df', 'isFraud']
drop_correlated_cols(master_df_merged, 0.9, cols_to_keep = cols_to_keep, sample_frac = 0.2)
master_df_merged.head()

,TransactionID,isFraud,TransactionDT,TransactionAmt_x,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1_x,dist2_x,P_emaildomain_x,R_emaildomain_x,C1_x,C5_x,C9_x,C13_x,D1_x,D3_x,D4_x,D5_x,D8_x,D9,D10_x,D11_x,D14_x,D15_x,M3_x,M4_x,M5_x,M6_x,M7_x,M8_x,M9_x,V4_x,V12_x,V20_x,V35_x,V53_x,V56_x,V62_x,V70_x,V75_x,V82_x,V87_x,V94,V99_x,V102,V112_x,V126_x,V127_x,V130_x,V141_x,V145_x,V149_x,V152_x,V162_x,V173_x,V176_x,V188_x,V189_x,V191_x,V201_x,V206_x,V208_x,V209_x,V223_x,V230_x,V244_x,V245_x,V254_x,V269_x,V271_x,V283_x,V285_x,V304_x,V307_x,V312_x,V313_x,V314_x,V320_x,V329,id_01_x,id_02_x,id_05_x,id_06_x,id_13_x,id_17_x,id_19_x,id_20_x,id_26_x,DeviceInfo_x,is_train_df,P_emaildomain_2_x,P_emaildomain_3_x,id_31_edge_x,id_31_chrome_version_newness_x,id_31_safari_version_newness_x,id_30_android_x,id_33_resolution_x,TransactionAmt_decimal_x,weekday,card3_144.0,card3_150.0,card3_185.0,R_emaildomain_2_com_x,id_14_-420.0_x,id_14_60.0_x,M4_M0_x,M4_M1_x,card6_credit,M8_F_x,hours_5.0,hours_11.0,card4_american express,R_emaildomain_anonymous.com_x,M6_F_x,M3_F_x,M5_F_x,card1_9500,card1_9633,card1_17188,card1_infrequent_category,P_emaildomain_comcast.net_x,P_emaildomain_gmail.com_x,P_emaildomain_hotmail.com_x,P_emaildomain_me.com_x,P_emaildomain_msn.com_x,P_emaildomain_optonline.net_x,P_emaildomain_verizon.net_x,P_emaildomain_infrequent_category_x,addr1_204.0,addr1_299.0,addr1_325.0,addr1_337.0,addr1_472.0,addr1_485.0,addr1_infrequent_category,ProductCD_H,ProductCD_R,id_20_333.0_x,id_20_549.0_x,id_20_612.0_x,card2_225.0,card2_268.0,card2_321.0,card2_481.0,card2_490.0,card2_553.0,card2_555.0,card2_567.0,card2_infrequent_category,M7_F_x,weekday_1.0,weekday_2.0,weekday_3.0,weekday_4.0,weekday_5.0,card5_166.0,card5_224.0,P_emaildomain_1_infrequent_category_x,id_17_166.0,id_32_24.0_x,P_emaildomain_y,R_emaildomain_y,C1_y,C2_y,C6_y,C8_y,C11_y,C13_y,C14_y,M3_y,M4_y,M5_y,M6_y,M7_y,M8_y,M9_y,V12_y,V35_y,V36_y,V53_y,V75_y,V76_y,V87_y,V91_y,V99_y,V126_y,V127_y,V128_y,V145_y,V152_y,V176_y,V177_y,V230_y,V257_y,V258_y,V304_y,V306_y,id_02_y,DeviceInfo_y,P_emaildomain_2_y,P_emaildomain_3_y,id_31_edge_y,id_30_android_y,TransactionAmt_decimal_y,R_emaildomain_2_com_y,id_14_-420.0_y,R_emaildomain_anonymous.com_y,M6_F_y,id_20_333.0_y,id_20_549.0_y,M7_F_y,id_32_24.0_y
0,2987000,0.0,86400,68.5,W,13926,NaN,150.0,discover,142.0,credit,315.0,87.0,19.0,NaN,NaN,NaN,1.0,0.0,1.0,1.0,14.0,13.0,NaN,NaN,NaN,NaN,13.0,13.0,NaN,0.0,T,M2,F,T,NaN,NaN,NaN,1.0,1.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,117.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,117.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,500,0.0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,0.452748,-56.367245,-35.609619,0.147989,-7.334560,-76.363113,-4.731055,T,M2,F,T,NaN,NaN,NaN,0.332354,NaN,NaN,0.311933,0.420110,0.371908,-0.400271,-0.477729,-5.658206,-442.928955,-6130.065430,-47.239407,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000325,-98.388420,NaN,NaN,NaN,NaN,NaN,NaN,100.971710,-0.015990,-0.019621,-0.003047,-0.080853,-0.006322,-0.000008,-0.390713,-0.016675
1,2987001,0.0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,credit,325.0,87.0,NaN,NaN,gmail.com,NaN,1.0,0.0,0.0,1.0,0.0,NaN,0.0,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,M0,T,T,NaN,NaN,NaN,NaN,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,com,NaN,NaN,NaN,NaN,NaN,NaN,0,0.0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,gmail.com,NaN,-3.620259,-56.254463,-4.153918,-0.184928,-9.372200,-2.555614,-0.118743,NaN,M0,T,T,NaN,NaN,NaN,-0.726845,-0.738347,-0.600466,-0.845680,-0.728800,-0.656142,-0.179304,-0.418249,-7.665982,-29.248983,-1

In [13]:
gc.collect()
master_df_merged.shape

(1097231, 219)

In [14]:
master_df = master_df_merged
del master_df_merged

In [15]:
for col in master_df.select_dtypes(exclude='number').columns:
    master_df[col] = master_df[col].astype('category').cat.codes

In [16]:
'''
length_ones = len(master_df[master_df['isFraud']==1])
train_balanced = pd.concat([master_df[master_df['isFraud']==1], (master_df[master_df['isFraud']==0]).sample(length_ones)], axis=0)

#train_balanced = train_balanced.sample(10000)


X_train, X_test, y_train, y_test = train_test_split(
    train_balanced.drop(columns=['isFraud', 'TransactionID', 'TransactionDT']), train_balanced['isFraud'], 
    test_size=1/4, stratify =train_balanced['isFraud'],  random_state=0)

print(X_train.shape)
print(X_test.shape)

clf = XGBClassifier(max_depth=5, n_estimators=1000, verbosity=1)
clf.fit(X_train, y_train)
pred_prob = clf.predict_proba(X_test)
pred_prob[:, 1]
roc_score = roc_auc_score(y_test, pred_prob[:, 1])
print("roc_auc score %.4f" % roc_score)
xgboost.plot_importance(clf, max_num_features=20, importance_type='gain')
xgboost.plot_importance(clf, max_num_features=20, importance_type='weight')
'''

'\nlength_ones = len(master_df[master_df[\'isFraud\']==1])\ntrain_balanced = pd.concat([master_df[master_df[\'isFraud\']==1], (master_df[master_df[\'isFraud\']==0]).sample(length_ones)], axis=0)\n\n#train_balanced = train_balanced.sample(10000)\n\n\nX_train, X_test, y_train, y_test = train_test_split(\n    train_balanced.drop(columns=[\'isFraud\', \'TransactionID\', \'TransactionDT\']), train_balanced[\'isFraud\'], \n    test_size=1/4, stratify =train_balanced[\'isFraud\'],  random_state=0)\n\nprint(X_train.shape)\nprint(X_test.shape)\n\nclf = XGBClassifier(max_depth=5, n_estimators=1000, verbosity=1)\nclf.fit(X_train, y_train)\npred_prob = clf.predict_proba(X_test)\npred_prob[:, 1]\nroc_score = roc_auc_score(y_test, pred_prob[:, 1])\nprint("roc_auc score %.4f" % roc_score)\nxgboost.plot_importance(clf, max_num_features=20, importance_type=\'gain\')\nxgboost.plot_importance(clf, max_num_features=20, importance_type=\'weight\')\n'

In [17]:
train_balanced = master_df[master_df['isFraud'].notnull()]

temp_list_to_drop = []
temp_list_to_drop.extend(['isFraud', 'TransactionID', 'TransactionDT', 'is_train_df'])

print(train_balanced.shape)


clf = XGBClassifier(max_depth=50)
clf.fit(train_balanced.drop(columns=temp_list_to_drop), train_balanced['isFraud'])

(590540, 219)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=50,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [18]:
gc.collect()

19

In [19]:
# prepare submission
temp_list_to_drop = []
#temp_list_to_drop = list(cols_cat)
temp_list_to_drop.extend(['isFraud', 'TransactionID', 'TransactionDT'])
temp_list_to_include = list(set(master_df.columns).difference(set(temp_list_to_drop)))

temp_list_to_drop = []
#temp_list_to_drop = list(cols_cat)
temp_list_to_drop.extend(['isFraud', 'TransactionID', 'TransactionDT'])

temp_list_to_include =  list(train_balanced.drop(columns=temp_list_to_drop).columns)

temp_list_to_drop = []
#temp_list_to_drop = list(cols_cat)
temp_list_to_drop.extend(['isFraud', 'TransactionID', 'TransactionDT', 'is_train_df'])

counter_from = master_df.loc[master_df['is_train_df']==0, 'isFraud'].index[0]

len_master_df = len(master_df)

print(counter_from)
print(len_master_df)
print('start!!')
while counter_from < len_master_df:
    print(counter_from)
    counter_to = counter_from + 10000
    pred = pd.DataFrame()
    #print(len(master_df['isFraud'].loc[counter_from:counter_to]))
    #print(len(master_df.loc[counter_from:counter_to, [col for col in master_df.columns if col not in temp_list_to_drop]]))
    master_df['isFraud'].loc[counter_from:counter_to] =  clf.predict_proba(master_df.loc[counter_from:counter_to, [col for col in master_df.columns if col not in temp_list_to_drop]])[:, 1]
    
    counter_from += 10000
    gc.collect()
#print(temp_list_to_include)

590540
1097231
start!!
590540
600540
610540
620540
630540
640540
650540
660540
670540
680540
690540
700540
710540
720540
730540
740540
750540
760540
770540
780540
790540
800540
810540
820540
830540
840540
850540
860540
870540
880540
890540
900540
910540
920540
930540
940540
950540
960540
970540
980540
990540
1000540
1010540
1020540
1030540
1040540
1050540
1060540
1070540
1080540
1090540


In [20]:
#sample_submission.head()

In [21]:
counter_from = master_df.loc[master_df['is_train_df']==0, 'isFraud'].index[0]
submission = pd.DataFrame(master_df[['TransactionID', 'isFraud']].loc[counter_from:]).reset_index(drop = True)
submission.head()

,TransactionID,isFraud
0,3663549,0.000312
1,3663550,0.000405
2,3663551,0.000931
3,3663552,0.001486
4,3663553,0.000265


In [22]:
submission.describe()

,TransactionID,isFraud
count,5.066910e+05,506691.000000
mean,3.916894e+06,0.020221
std,1.462692e+05,0.116496
min,3.663549e+06,0.000059
25%,3.790222e+06,0.000535
50%,3.916894e+06,0.001096
75%,4.043566e+06,0.002787
max,4.170239e+06,0.999529


In [23]:
submission.to_csv('submission.csv', index=False)